# Product-Based Capstone Project

Team ID : C22-PS096

Team Member :
* (ML) M7134F1606 - Muhammad Fadli Ramadhan - Politeknik Negeri Sriwijaya
* (ML) M2008F0851 - Adhitya Ghiffari Pramudito - Universitas Gadjah Mada
* (MD) A2191F1821 - Ahmad Ansori Palembani - Universitas Bina Darma
* (MD) A7191F1820 - Muhammad Fharid Akbar - Universitas Bina Darma
* (CC) C2322F2819 - Muhammad Mustafa Kamal  - Universitas Syiah Kuala
* (CC) C7457F3068 - Wulan Ayu Rania Sari  -Universitas Nahdlatul Ulama Lampung

Final Selected Themes :
Human Healthcare & Animal Welfare

Title of the Project :
SkinCan : Skin Cancer Detection App



**Executive Summary :**

At this time there are so many dangers that can lurk human health, even in this era of pandemic, maintaining health is very important for everyone. People should have awareness to maintain health, especially the health of the skin. Therefore, by utilizing machine learning, cloud computing to create a skin cancer detection application by identifying the input image to remind the public the importance of maintaining healthy skin. In this project, We have a problem formulation as follows:
* How to detect various skin cancers in humans through the application?
* How to treat skin cancer early?
* How accurately does this application detect skin cancer? 

Our team wants to solve this problem because we want someone who has the potential for cancer to avoid a more serious problem, so that users who use this application can take preventive measures to prevent this problem.

**How did your team come up with this project?**

We want to build an application that can help the public and health workers to detect skin diseases in humans, including skin cancer. This application uses Deep Learning algorithms to predict various skin diseases to achieve maximum accuracy in predicting skin diseases at an early stage. Feature extraction plays a key role in the classification of skin diseases. In addition, Explainable AI is used to interpret decisions made by our machine learning models. With this application, hopefully, it can help users to detect skin diseases early so they can take preventive measures.

**Based on your knowledge and explorations, tell us the Machine Learning Part of your capstone?**

In this project, we’re using Python as main programming language, Tensorflow as mandatory library, Sckit-learn and keras as libraries, AI Platform as platform, TFLite as Tools for Model Deployment to Android, Kaggle as dataset resource, Google Colab as IDE, and Machine Learning Model as main the feature in our app.


# **Modeling Machine Learning Model for SkinCan App**

# Data Collecting

In [1]:
import warnings # tf needs to learn to stfu
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)
warnings.simplefilter(action="ignore", category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

In [2]:
# download plotly package
!pip install plotly

In [3]:
# import libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import io
import os

from PIL import Image
from glob import glob
import itertools

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import seaborn as sns


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, BatchNormalization, Dropout, Dense, MaxPool2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

from IPython.display import display
#To see the value of multiple statements at once.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"



##  **Create Function**

**Train test splits**

In [4]:
# define function for train test splits
def train_test_splits(X, Y):
    # Splitting into train and test set
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=1)
    
    # Prepare data for training and testing the model
    train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 10,
                                  width_shift_range = 0.2,
                                  height_shift_range = 0.2,
                                  shear_range = 0.2,
                                  horizontal_flip = True,
                                  vertical_flip = True,
                                  fill_mode = 'nearest')
    train_datagen.fit(X_train)
    test_datagen = ImageDataGenerator(rescale = 1./255)
    test_datagen.fit(X_test)
    return X_train, X_test, Y_train, Y_test

In [5]:
# define function for create model
def model():
    model = Sequential()
    model.add(Conv2D(16, kernel_size = (3,3), input_shape = (28, 28, 3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2)))

    model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))
    model.add(Conv2D(128, kernel_size = (3,3), activation = 'relu', padding = 'same'))
    model.add(MaxPool2D(pool_size = (2,2), padding = 'same'))

    model.add(Flatten())
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(7, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)

    model.compile(loss = 'sparse_categorical_crossentropy',
                 optimizer = optimizer,
                  metrics = ['accuracy'])
    print(model.summary())
    return model;

**Training model**

In [6]:
# Define function for training model
def train_model(model, X_train, Y_train, EPOCHS=25):
    early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, 
                           mode='auto')
                               
    
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, 
                              verbose=1, mode='auto')
    
    history = model.fit(X_train,
                        Y_train,
                        validation_split=0.2,
                        batch_size = 64,
                        epochs = EPOCHS,
                        callbacks = [reduce_lr, early_stop])
    return history

**Test model**

In [7]:
# Define function for model testing
def test_model(model, X_test, Y_test):
    model_acc = model.evaluate(X_test, Y_test, verbose=0)[1]
    print("Test Accuracy: {:.3f}%".format(model_acc * 100))
    y_true = np.array(Y_test)
    y_pred = model.predict(X_test)
    y_pred = np.array(list(map(lambda x: np.argmax(x), y_pred)))
    clr = classification_report(y_true, y_pred, target_names=label_mapping.values())
    print(clr)
    
    sample_data = X_test[:15]
    plt.figure(figsize=(22, 12))
    for i in range(15):
        plt.subplot(3, 5, i + 1)
        plt.imshow(sample_data[i])
        plt.title(label_mapping[y_true[i][0]] + '|' + label_mapping[y_pred[i]])
        plt.axis("off")
    plt.show() 

**Training curves plot**

In [8]:
# Define function for training curves plot
def plot_model_training_curve(history):
    fig = make_subplots(rows=1, cols=2, subplot_titles=['Model Accuracy', 'Model Loss'])
    fig.add_trace(
        go.Scatter(
            y=history.history['accuracy'], 
            name='train_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_accuracy'], 
            name='val_acc'), 
        row=1, col=1)
    fig.add_trace(
        go.Scatter(
            y=history.history['loss'], 
            name='train_loss'), 
        row=1, col=2)
    fig.add_trace(
        go.Scatter(
            y=history.history['val_loss'], 
            name='val_loss'), 
        row=1, col=2)
    fig.show()

**Multi class confision matrix**

In [9]:
# Define function for create multi class confision matrix
def create_confusion_matrix(model, x_test_normalized, y_test, cm_plot_labels, name):
    #Get the true and predicted labels
    y_predict_classes, y_true_classes = cal_true_pred_classes(model, x_test_normalized, y_test)

    #Compute the confusion matrix
    confusion_matrix_computed = confusion_matrix(y_true_classes, y_predict_classes)

    #Plot the confusion matrix
    plot_confusion_matrix(confusion_matrix_computed, cm_plot_labels, name) 

In [10]:
# Define function for confusion matrix plot
def plot_confusion_matrix(cm, classes,
                          name,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    
    plt.figure(figsize=(8,6))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(name)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    
#     b, t = plt.ylim()
#     b += 0.5
#     t -= 0.5
#     plt.ylim(b, t)
    
    plt.tight_layout()
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    fig = plt
    fig.savefig('E:\SIB BANGKIT 2022\CAPSTONE\Model Aset\\' + name + '_confusion_matrix.png', dpi=300)